In [4]:
import pandas as pd
import numpy as np
import pandas_ta as pta
import ta
# from datapackage import Package
# from selenium import webdriver
# import requests
# from bs4 import BeautifulSoup
import yfinance as yf
import os
import random
import shutil
import time
import datetime as dt
from tensorflow.keras.layers import LSTM
import tensorflow as tf
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from numpy import array
from tensorflow.keras.utils import plot_model
import graphviz
import pydot
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Dropout

In [2]:
# PARAMETERS

steps = 20
epochs = 50
nodes_per_layer = 64
shift = -1
batch_size = 20
loss_function = "mean_squared_error"
dropout_rate = 0.2

In [3]:
file = "NVDA.csv"
ticker_code = file.replace(".csv", "")
dataset = pd.read_csv("data/" + file)
shift = -1
dataset = dataset.drop(columns=["Date", "Dividends", "Stock Splits"])
dataset = dataset.dropna()
train_set_len = int(len(dataset) * 0.8)
train_set = dataset[:train_set_len]
test_set = dataset[train_set_len:]
X_train = train_set.drop(columns=["Close"])[:shift]
y_train = train_set["Close"].shift(shift).dropna()
X_test = test_set.drop(columns=["Close"])[:shift]
y_test = test_set["Close"].shift(shift).dropna()
x_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()
X_train_scaled = x_scaler.fit_transform(X_train)
y_train_scaled = y_scaler.fit_transform(np.array(y_train).reshape(-1,1))
X_test_scaled = x_scaler.transform(X_test)
y_test_scaled = y_scaler.transform(np.array(y_test).reshape(-1,1))
pd.to_pickle(x_scaler, "scalers/" + str(ticker_code) + "_x_scaler.pkl")
pd.to_pickle(y_scaler, "scalers/" + str(ticker_code) + "_y_scaler.pkl")
pd.to_pickle(train_set, "train_sets/" + ticker_code + "_train_set.pkl")
pd.to_pickle(test_set, "test_sets/" + ticker_code + "_test_set.pkl")
pd.to_pickle(X_train, "sets/X_train/" + ticker_code + "_X_train.pkl")
pd.to_pickle(X_test, "sets/X_test/" + ticker_code + "_X_test.pkl")
pd.to_pickle(y_train, "sets/y_train/" + ticker_code + "_y_train.pkl")
pd.to_pickle(y_test, "sets/y_test/" + ticker_code + "_y_test.pkl")
pd.to_pickle(X_train_scaled, "sets/X_train/" + ticker_code + "_X_train_scaled.pkl")
pd.to_pickle(X_test_scaled, "sets/X_test/" + ticker_code + "_X_test_scaled.pkl")
pd.to_pickle(y_train_scaled, "sets/y_train/" + ticker_code + "_y_train_scaled.pkl")
pd.to_pickle(y_test_scaled, "sets/y_test/" + ticker_code + "_y_test_scaled.pkl")

In [4]:
X_train_scaled = pd.read_pickle("sets/X_train/" + ticker_code + "_X_train_scaled.pkl")
y_train_scaled = pd.read_pickle("sets/y_train/" + ticker_code + "_y_train_scaled.pkl")
X_train_list = []
for i in range(steps, len(X_train_scaled)):
	X_train_list.append(X_train_scaled[i-steps:i])
X_train_list = np.array(X_train_list)
model = Sequential()
model.add(LSTM(nodes_per_layer, activation='relu', input_shape=(steps, X_train_list.shape[2]), return_sequences=True))
model.add(Dropout(dropout_rate))
model.add(LSTM(nodes_per_layer, return_sequences=True))
model.add(Dropout(dropout_rate))
model.add(LSTM(nodes_per_layer, return_sequences=True))
model.add(Dropout(dropout_rate))
model.add(LSTM(nodes_per_layer, return_sequences=True))
model.add(Dropout(dropout_rate))
model.add(LSTM(nodes_per_layer, return_sequences=True))
model.add(Dropout(dropout_rate))
model.add(LSTM(nodes_per_layer, return_sequences=True))
model.add(Dropout(dropout_rate))
model.add(LSTM(nodes_per_layer, return_sequences=False))
model.add(Dense(units=1, activation='relu'))
model.compile(optimizer='adam', loss=loss_function)
model.fit(X_train_list, y_train_scaled[:(steps * shift)], epochs=epochs, batch_size=batch_size)
X_test_scaled = pd.read_pickle("sets/X_test/" + ticker_code + "_X_test_scaled.pkl")
y_test_scaled = pd.read_pickle("sets/y_test/" + ticker_code + "_y_test_scaled.pkl")
X_test_list = []
for i in range(steps, len(X_test_scaled)):
	X_test_list.append(X_test_scaled[i-steps:i])
X_test_list = np.array(X_test_list)
score = model.evaluate(X_test_list, y_test_scaled[:(steps * -1)])
print(f"{ticker_code} processing finished")
pd.to_pickle(model, "models/" + ticker_code + "_score_" + str(round(score, 2)) + ".pkl")

Epoch 1/50
240/240 [==============================] - 18s 36ms/step - loss: 0.0024
Epoch 2/50
240/240 [==============================] - 10s 41ms/step - loss: 3.0902e-04
Epoch 3/50
240/240 [==============================] - 9s 36ms/step - loss: 2.9453e-04
Epoch 4/50
240/240 [==============================] - 9s 36ms/step - loss: 2.3366e-04
Epoch 5/50
240/240 [==============================] - 9s 36ms/step - loss: 3.0467e-04
Epoch 6/50
240/240 [==============================] - 9s 35ms/step - loss: 2.5783e-04
Epoch 7/50
240/240 [==============================] - 9s 36ms/step - loss: 3.8368e-04
Epoch 8/50
240/240 [==============================] - 9s 36ms/step - loss: 1.7940e-04
Epoch 9/50
240/240 [==============================] - 9s 36ms/step - loss: 1.3837e-04
Epoch 10/50
240/240 [==============================] - 9s 36ms/step - loss: 1.8957e-04
Epoch 11/50
240/240 [==============================] - 9s 36ms/step - loss: 2.3579e-04
Epoch 12/50
240/240 [==============================] -

In [13]:
time.time()

1669814621.9930823

In [5]:
from datetime import timedelta, time

ticker_code = "MSFT"
ticker = yf.Ticker(ticker_code.replace(".", "-"))
twenty_days_ago = np.busday_offset(dates=dt.date.today(), offsets=-20, roll='backward')
raw_input = ticker.history(start=str(twenty_days_ago))
# if len(input) != 20:
# 	print("ERROR")
# len(input)
original_input = raw_input.reset_index().drop(columns=["Date", "Dividends", "Stock Splits", "Close"])


In [16]:
top_25 = ["AAPL", "MSFT", "AMZN", "TSLA", "GOOGL", "BRK.B", "UNH", "JNJ", "XOM", "JPM", "META", "V", "PG", "NVDA", "HD", "CVX", "LLY", "MA", "ABBV", "PFE", "MRK", "PEP", "BAC", "KO"]
list_of_prices = []
print(os.getcwd())
for i in top_25:
	ticker = yf.Ticker(i.replace(".", "-"))
	hist = ticker.history(end = dt.date.today(), start = str(np.busday_offset(dates=dt.date.today(), offsets=-20, roll='backward')))
	if (os.path.isdir(r"D:\Documents\Ironhack\Final Project\Flask\recent_history") == False):
		os.mkdir(r"D:\Documents\Ironhack\Final Project\Flask\recent_history")
	for csv in os.listdir(r"D:\Documents\Ironhack\Final Project\Flask\recent_history"):
		os.remove(csv)
	pd.DataFrame(hist).to_csv("recent_history/" + i + "_last_20_days.csv")
	list_of_prices.append({"Stock":i, "Price":round(hist["Close"][len(hist) - 1], 2)})
price_df = pd.DataFrame()
for price in list_of_prices:
	#print(pd.DataFrame([price]))
	pd.concat([price_df,[price.keys(), price.values()]], axis=0, ignore_index=1)
	display(price_df)
price_df

d:\Documents\Ironhack\Final Project


TypeError: cannot concatenate object of type '<class 'list'>'; only Series and DataFrame objs are valid

In [200]:
sc = pd.read_pickle("scalers/" + ticker_code + "_x_scaler.pkl")
input = sc.transform(original_input)
input.shape
input = input.reshape(-1, input.shape[0], input.shape[1])

In [201]:
def get_model(stock):
	for file in os.listdir("models"):
		if file.startswith(stock):
			print("Reading file... - " + file)
			return pd.read_pickle("models/" + file)

model = get_model(ticker_code)

Reading file... - MSFT_score_10.59.pkl
Keras model archive loading:
File Name                                             Modified             Size
config.json                                    2022-11-29 11:50:26         7695
metadata.json                                  2022-11-29 11:50:26           64
variables.h5                                   2022-11-29 11:50:26      2662184
Keras weights file (<HDF5 file "variables.h5" (mode r)>) loading:
...layers\dense
......vars
.........0
.........1
...layers\dropout
......vars
...layers\dropout_1
......vars
...layers\dropout_2
......vars
...layers\dropout_3
......vars
...layers\dropout_4
......vars
...layers\dropout_5
......vars
...layers\lstm
......vars
...layers\lstm\cell
......vars
.........0
.........1
.........2
...layers\lstm_1
......vars
...layers\lstm_1\cell
......vars
.........0
.........1
.........2
...layers\lstm_2
......vars
...layers\lstm_2\cell
......vars
.........0
.........1
.........2
...layers\lstm_3
......vars
...laye

In [202]:
y_sc = pd.read_pickle("scalers/" + ticker_code + "_y_scaler.pkl")
y_sc.inverse_transform(model.predict(input))

1/1 [==============================] - 2s 2s/step


array([[35.117893]], dtype=float32)

In [205]:
X_test_scaled = pd.read_pickle("sets/X_test/" + ticker_code + "_X_test_scaled.pkl")
y_test_scaled = pd.read_pickle("sets/y_test/" + ticker_code + "_y_test_scaled.pkl")
X_test_scaled = X_test_scaled.reshape(X_test_scaled.shape[0], X_test_scaled.shape[1], -1)
steps = 20
X_test_list = []
for i in range(steps, len(X_test_scaled)):
	X_test_list.append(X_test_scaled[i-steps:i])
X_test_list = np.array(X_test_list)
model.evaluate(X_test_list, y_test_scaled[:(steps * -1)])

58/58 [==============================] - 4s 28ms/step - loss: 10.5947


10.594701766967773

In [204]:
raw_input

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2022-11-02 00:00:00-04:00,228.815173,230.649998,219.421632,219.481476,38407000,0.00,0
2022-11-03 00:00:00-04:00,219.471488,219.790596,213.378658,213.647903,36633900,0.00,0
2022-11-04 00:00:00-04:00,216.938633,220.967272,212.830200,220.767838,36767800,0.00,0
2022-11-07 00:00:00-05:00,221.366164,227.768121,220.658153,227.229630,33498000,0.00,0
2022-11-08 00:00:00-05:00,228.057301,230.999008,225.205338,228.226822,28192500,0.00,0
2022-11-09 00:00:00-05:00,226.731038,227.987506,223.699587,223.879074,27852900,0.00,0
2022-11-10 00:00:00-05:00,234.768379,242.646187,234.339595,242.297165,46268000,0.00,0
2022-11-11 00:00:00-05:00,242.307141,247.293090,241.250107,246.415558,34600900,0.00,0
2022-11-14 00:00:00-05:00,241.309951,243.224554,238.537765,240.871185,31123300,0.00,0
